# Callbacks

Every tool in the `mpltoolbox` comes with entry points for adding callbacks to different events.
The different events are:

- `on_create`: called when drawing the shape (rectangle, line or polygon) is complete
- `on_change`: called when the shape has changed in any way (position, size...)
- `on_remove`: called when the shape is deleted (middle-click, or Ctrl + left-click)
- `on_vertex_press`: called when a vertex is clicked (left-click)
- `on_vertex_move`: called when a vertex is moved
- `on_vertex_release`: called when the mouse button is released after clicking a vertex
- `on_drag_press`: called when the entire shape (rectangle, line, etc..) is right clicked to initiate drag
- `on_drag_move`: called for every movement during shape drag (right-click and hold)
- `on_drag_release`: called when the shape is released after drag

Below is a couple of examples on how these callbacks are used.

In [ ]:
%matplotlib widget
import numpy as np
import mpltoolbox as tbx
import matplotlib.pyplot as plt

plt.ioff()

## Example 1: Add markers to slice 3d cube

We first make some three-dimensional data:

In [ ]:
N = 200
M = 300
L = 100
xx = np.arange(N, dtype=np.float64)
yy = np.arange(M, dtype=np.float64)
zz = np.arange(L, dtype=np.float64)
x, y, z = np.meshgrid(xx, yy, zz, indexing="ij")
b = N / 20.0
c = M / 2.0
d = L / 2.0
r = np.sqrt(((x - c) / b) ** 2 + ((y - c) / b) ** 2 + ((z - d) / b) ** 2)
a = 10.0 * np.sin(r) + np.random.random([N, M, L])

Create a figure to display the first `z` slice of the data as a two-dimensional image,
as well as an empty subplot below:

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(7, 7))
fig.canvas.header_visible = False
ax[0].imshow(a[..., 0], interpolation="none", origin="lower")
ax[0].set(xlabel='x', ylabel='y')
ax[1].set(xlabel='z')
fig.tight_layout()

Then we add a `Points` tool where:

- When a dot is added on the image, a line is created in the lower panel, showing a one-dimensional `z` slice at the location of the marker
- When a dot is moved, the `z` line is updated accordingly
- When a dot is removed, remove the corresponding `z` profile

In [ ]:
def make_line(new_point):
    (line,) = ax[1].plot(a[int(new_point.y), int(new_point.x), :])
    new_point.associated = line


def update_line(new_point):
    new_point.associated.set_ydata(a[int(new_point.y), int(new_point.x), :])


def remove_line(point):
    point.associated.remove()


points = tbx.Points(ax=ax[0], mec="white")
points.on_create(make_line)
points.on_change(update_line)
points.on_remove(remove_line)

In [ ]:
points.click(x=101, y=67)
points.click(x=116, y=96)
points.click(x=271, y=155)

In [ ]:
fig

## Example 2: Histogram inside a rectangular region

In the second example, we use the `Ractangles` tool to draw rectangles on the same 2d image.
This defines a region of interest, inside which the data will be histogrammed and displayed on the lower panel. 

In [ ]:
fig2, ax2 = plt.subplots(2, 1, figsize=(7, 7))
fig2.canvas.header_visible = False
fig2.tight_layout()
ax2[0].imshow(a[..., 0], interpolation="none", origin="lower")


def make_hist(new_rectangle):
    xy = new_rectangle.xy
    ix0 = int(xy[0])
    iy0 = int(xy[1])
    ix1 = int(xy[0] + new_rectangle.width)
    iy1 = int(xy[1] + new_rectangle.height)
    n, bins, patches = ax2[1].hist(
        a[min(iy0, iy1) : max(iy0, iy1), min(ix0, ix1) : max(ix0, ix1), :].ravel(),
        edgecolor=new_rectangle.edgecolor,
        facecolor="None",
    )
    new_rectangle.associated = patches


def update_hist(new_rectangle):
    new_rectangle.associated.remove()
    make_hist(new_rectangle)


def remove_hist(rectangle):
    rectangle.associated.remove()


rects = tbx.Rectangles(ax=ax2[0], facecolor=(0, 0, 0, 0.3))
rects.on_create(make_hist)
rects.on_change(update_hist)
rects.on_remove(remove_hist)

In [ ]:
rects.click(x=51, y=67)
rects.click(x=159, y=125)

rects.click(x=200, y=130)
rects.click(x=260, y=175)

In [ ]:
fig2

## Example 3: Multiple callbacks

It is also possible to add multiple callbacks by calling the `on_*` methods multiple times:

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(7, 7))
fig.canvas.header_visible = False
ax[0].set(xlabel='x', ylabel='y')
ax[1].set(xlabel='z')
fig.tight_layout()
ax[0].imshow(a[..., 0], interpolation="none", origin="lower")


def make_line(p):
    (line,) = ax[1].plot(a[int(p.y), int(p.y), :])
    p.associated = line


def update_line(p):
    p.associated.set_ydata(a[int(p.y), int(p.y), :])


def make_text(p):
    t = ax[1].text(50.0, np.random.random() * 10, f"x={int(p.x)}; y={int(p.y)}")
    p.text = t


def update_text(p):
    p.text.set_text(f"x={int(p.x)}; y={int(p.y)}")


points = tbx.Points(ax=ax[0], mec="white")
points.on_create(make_line)
points.on_create(make_text)
points.on_change(update_line)
points.on_change(update_text)

In [ ]:
points.click(x=101, y=67)
points.click(x=116, y=96)
points.click(x=271, y=155)

In [ ]:
fig